Docker pure python 3.11 build (DataSphere 1 GPU A100 • 28 vCPUs • 119 ГБ RAM1 GPU A100 • 28 vCPUs • 119 ГБ RAM)
```
FROM ubuntu:22.04
ENV DEBIAN_FRONTEND=noninteractive

RUN apt-get update && apt-get install -y --no-install-recommends \
      ca-certificates \
      python3.11 python3.11-venv python3.11-distutils \
      python3-pip \
    && ln -sf /usr/bin/python3.11 /usr/bin/python3 \
    && ln -sf /usr/bin/pip3 /usr/bin/pip \
    && python3 -m pip install -U pip \
    && rm -rf /var/lib/apt/lists/*

RUN useradd -ms /bin/bash --uid 1000 jupyter
USER jupyter
WORKDIR /home/jupyter
```

In [3]:
!python3 --version

Python 3.11.0rc1


In [4]:
%pip install murmurhash==1.0.13
%pip install numpy==2.1.2
%pip install tensorboard==2.20.0
%pip install transformers==4.50.3

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
%pip install torch==2.5.1+cu121 --index-url https://download.pytorch.org/whl/cu121

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu121

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [6]:
import json

import torch
from torch.utils.data import DataLoader

from modeling import utils
from modeling.dataloader import BatchProcessor
from modeling.dataset import Dataset
from modeling.loss import BCELoss
from modeling.metric import NDCGMetric, RecallMetric
from modeling.models import SasRecModel
from modeling.trainer import Trainer
from modeling.utils import parse_args, create_logger, fix_random_seed

/home/jupyter/.local/lib/python3.11/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/kernel/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/kernel/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [7]:
def parse_config(config_path):
    with open(config_path) as f:
        params = json.load(f)
    
    return params

In [9]:
config = parse_config('configs/sasrec_train_config.json')
print('Training config: \n{}'.format(json.dumps(config, indent=2)))

Training config: 
{
  "experiment_name": "sasrec_beauty",
  "dataset": {
    "inter_json_path": "../data/Beauty/inter.json",
    "max_sequence_length": 20,
    "sampler_type": "sasrec"
  },
  "dataloader": {
    "train_batch_size": 256,
    "validation_batch_size": 256
  },
  "model": {
    "embedding_dim": 64,
    "num_heads": 2,
    "num_layers": 2,
    "dim_feedforward": 256,
    "dropout": 0.3,
    "activation": "relu",
    "top_k": 20,
    "layer_norm_eps": 1e-08,
    "initializer_range": 0.02
  },
  "optimizer": {
    "lr": 0.001
  }
}


In [10]:
SEED_VALUE = 42
fix_random_seed(SEED_VALUE)

In [11]:
print('Current DEVICE: {}'.format(utils.DEVICE))

Current DEVICE: cuda


In [12]:
dataset = Dataset.create(
    inter_json_path=config['dataset']['inter_json_path'],
    max_sequence_length=config['dataset']['max_sequence_length'],
    sampler_type=config['dataset']['sampler_type'],
    is_extended=False
)
dataset_num_items = dataset.num_items
dataset_max_sequence_length = dataset.max_sequence_length

train_sampler, validation_sampler, test_sampler = dataset.get_samplers()

[2025-10-20 18:49:17] [INFO]: Train dataset size: 22363
[2025-10-20 18:49:17] [INFO]: Validation dataset size: 22363
[2025-10-20 18:49:17] [INFO]: Test dataset size: 22363
[2025-10-20 18:49:17] [INFO]: Max item id: 12100


In [13]:
batch_processor = BatchProcessor()

train_dataloader = DataLoader(
    dataset=train_sampler,
    batch_size=config['dataloader']['train_batch_size'],
    drop_last=True,
    shuffle=True,
    collate_fn=batch_processor
)

validation_dataloader = DataLoader(
    dataset=validation_sampler,
    batch_size=config['dataloader']['validation_batch_size'],
    drop_last=False,
    shuffle=False,
    collate_fn=batch_processor
)

eval_dataloader = DataLoader(
    dataset=test_sampler,
    batch_size=config['dataloader']['validation_batch_size'],
    drop_last=False,
    shuffle=False,
    collate_fn=batch_processor
)

model = SasRecModel(
    num_items=dataset_num_items,
    max_sequence_length=dataset_max_sequence_length,
    embedding_dim=config['model']['embedding_dim'],
    num_heads=config['model']['num_heads'],
    num_layers=config['model']['num_layers'],
    dim_feedforward=config['model']['dim_feedforward'],
    activation=utils.get_activation_function(config['model']['activation']),
    topk_k=config['model']['top_k'],
    dropout=config['model']['dropout'],
    layer_norm_eps=config['model']['layer_norm_eps'],
    initializer_range=config['model']['initializer_range']
).to(utils.DEVICE)

In [14]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Overall parameters: {total_params:,}')
print(f'Trainable parameters: {trainable_params:,}')

Overall parameters: 875,712
Trainable parameters: 875,712


In [15]:
loss_function = BCELoss(
    positive_prefix='positive_scores',
    negative_prefix='negative_scores',
    output_prefix='loss'
)

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=config['optimizer']['lr'],
)

ranking_metrics = {
    'ndcg@5': NDCGMetric(5),
    'ndcg@10': NDCGMetric(10),
    'ndcg@20': NDCGMetric(20),
    'recall@5': RecallMetric(5),
    'recall@10': RecallMetric(10),
    'recall@20': RecallMetric(20)
}

print('Everything is ready for training process!')

Everything is ready for training process!


In [16]:
trainer = Trainer(
    experiment_name=config['experiment_name'],
    train_dataloader=train_dataloader,
    validation_dataloader=validation_dataloader,
    eval_dataloader=eval_dataloader,
    model=model,
    optimizer=optimizer,
    loss_function=loss_function,
    ranking_metrics=ranking_metrics,
    epoch_cnt=config.get('train_epochs_num'),
    step_cnt=config.get('train_steps_num'),
    best_metric='ndcg@20',
    epochs_threshold=config.get('early_stopping_threshold', 40),
    valid_step=256,
    eval_step=256
)

best_checkpoint = trainer.train()
trainer.save()

print('Training finished!')

[2025-10-20 18:49:33] [DEBUG]: Start training...
[2025-10-20 18:49:33] [DEBUG]: Start epoch 0
[2025-10-20 18:49:35] [DEBUG]: Running validation on step 0...
[2025-10-20 18:49:36] [DEBUG]: Running validation on step 0 is done!
[2025-10-20 18:49:36] [DEBUG]: Running eval on step 0...
[2025-10-20 18:49:37] [DEBUG]: Running eval on step 0 is done!


0 0.0005317691558482997 0 0.0003777147512159146


[2025-10-20 18:49:39] [DEBUG]: Start epoch 1
[2025-10-20 18:49:40] [DEBUG]: Start epoch 2
[2025-10-20 18:49:42] [DEBUG]: Running validation on step 256...
[2025-10-20 18:49:42] [DEBUG]: Running validation on step 256 is done!
[2025-10-20 18:49:42] [DEBUG]: Running eval on step 256...
[2025-10-20 18:49:43] [DEBUG]: Running eval on step 256 is done!
[2025-10-20 18:49:43] [DEBUG]: Start epoch 3


256 0.01679523827932295 0.0005317691558482997 0.014035641170889226


[2025-10-20 18:49:45] [DEBUG]: Start epoch 4
[2025-10-20 18:49:47] [DEBUG]: Start epoch 5
[2025-10-20 18:49:48] [DEBUG]: Running validation on step 512...
[2025-10-20 18:49:49] [DEBUG]: Running validation on step 512 is done!
[2025-10-20 18:49:49] [DEBUG]: Running eval on step 512...
[2025-10-20 18:49:49] [DEBUG]: Running eval on step 512 is done!


...

1792 0.04023630190704822 0.04002031452171876 0.034384775904968815


[2025-10-20 18:50:21] [DEBUG]: Start epoch 21
[2025-10-20 18:50:23] [DEBUG]: Start epoch 22
[2025-10-20 18:50:24] [DEBUG]: Start epoch 23
[2025-10-20 18:50:25] [DEBUG]: Running validation on step 2048...
[2025-10-20 18:50:26] [DEBUG]: Running validation on step 2048 is done!
[2025-10-20 18:50:26] [DEBUG]: Running eval on step 2048...
[2025-10-20 18:50:26] [DEBUG]: Running eval on step 2048 is done!
[2025-10-20 18:50:27] [DEBUG]: Start epoch 24
[2025-10-20 18:50:29] [DEBUG]: Start epoch 25
[2025-10-20 18:50:31] [DEBUG]: Start epoch 26
[2025-10-20 18:50:31] [DEBUG]: Running validation on step 2304...
[2025-10-20 18:50:32] [DEBUG]: Running validation on step 2304 is done!
[2025-10-20 18:50:32] [DEBUG]: Running eval on step 2304...
[2025-10-20 18:50:32] [DEBUG]: Running eval on step 2304 is done!


2304 0.04053881879220766 0.04023630190704822 0.034471569483120156


[2025-10-20 18:50:33] [DEBUG]: Start epoch 27
[2025-10-20 18:50:35] [DEBUG]: Start epoch 28
[2025-10-20 18:50:37] [DEBUG]: Start epoch 29
[2025-10-20 18:50:38] [DEBUG]: Running validation on step 2560...
[2025-10-20 18:50:38] [DEBUG]: Running validation on step 2560 is done!
[2025-10-20 18:50:38] [DEBUG]: Running eval on step 2560...
[2025-10-20 18:50:39] [DEBUG]: Running eval on step 2560 is done!
[2025-10-20 18:50:40] [DEBUG]: Start epoch 30
[2025-10-20 18:50:41] [DEBUG]: Start epoch 31
[2025-10-20 18:50:43] [DEBUG]: Start epoch 32
[2025-10-20 18:50:44] [DEBUG]: Running validation on step 2816...
[2025-10-20 18:50:44] [DEBUG]: Running validation on step 2816 is done!
[2025-10-20 18:50:44] [DEBUG]: Running eval on step 2816...
[2025-10-20 18:50:45] [DEBUG]: Running eval on step 2816 is done!
[2025-10-20 18:50:46] [DEBUG]: Start epoch 33
[2025-10-20 18:50:47] [DEBUG]: Start epoch 34
[2025-10-20 18:50:49] [DEBUG]: Start epoch 35
[2025-10-20 18:50:49] [DEBUG]: Running validation on step 

Training finished!


In [17]:
trainer.load(best_checkpoint)
trainer.eval()

[2025-10-20 18:52:39] [DEBUG]: Running eval on step 0...
[2025-10-20 18:52:39] [DEBUG]: Running eval on step 0 is done!


ndcg@5 0.020869553694170435
ndcg@10 0.0271735891299326
ndcg@20 0.034471569483120156
recall@5 0.031972454500737824
recall@10 0.051647811116576486
recall@20 0.08071367884451996
